# C++ Builder & Runner

This notebook generates a specific build directory, creates an SConstruct file inside it, and compiles the project.

In [1]:
import os
import urllib.request
import sys

def project_tdd_setup(project_name):
    # --- Настройка путей ---
    root_dir = os.path.abspath(project_name)
    src_dir = os.path.join(root_dir, "src")
    inc_dir = os.path.join(root_dir, "include")
    doctest_dir = os.path.join(inc_dir, "doctest")
    
    cpp_filename = "core.cpp"
    cpp_path = os.path.join(src_dir, cpp_filename)
    
    print(f"🚀 Initializing project: {project_name}")

    # 1. Создание структуры папок
    for d in [root_dir, src_dir, inc_dir, doctest_dir]:
        os.makedirs(d, exist_ok=True)
        print(f"📂 Created: {d}")

    # 2. Скачивание библиотек (Header-only)
    libs = {
        os.path.join(doctest_dir, "doctest.h"): "https://raw.githubusercontent.com/doctest/doctest/master/doctest/doctest.h",
        os.path.join(inc_dir, "nanobench.h"): "https://raw.githubusercontent.com/martinus/nanobench/master/src/include/nanobench.h"
    }

    print("⬇️ Downloading dependencies...")
    for path, url in libs.items():
        if not os.path.exists(path):
            try:
                print(f"   Fetching {os.path.basename(path)}...")
                urllib.request.urlretrieve(url, path)
            except Exception as e:
                print(f"❌ Error downloading {os.path.basename(path)}: {e}")
                print("   ⚠️ Please download manually and place in include folder.")

    # 3. Генерация .clang-format
    clang_format_content = """---
Language:        Cpp
BasedOnStyle:    Google
IndentWidth:     4
ColumnLimit:     120
SortIncludes:    true
FixNamespaceComments: true
AlignAfterOpenBracket: Align
AllowShortFunctionsOnASingleLine: Empty
BreakBeforeBraces: Attach
...
"""
    with open(os.path.join(root_dir, ".clang-format"), "w") as f:
        f.write(clang_format_content)
    print("✨ Generated .clang-format")

    # 4. Генерация .clang-tidy
    clang_tidy_content = """---
Checks:          'clang-diagnostic-*,clang-analyzer-*,bugprone-*,modernize-*,performance-*,readability-*,portability-*,-modernize-use-trailing-return-type'
WarningsAsErrors: '*'
CheckOptions:
  - key:             readability-identifier-naming.ClassCase
    value:           CamelCase
...
"""
    with open(os.path.join(root_dir, ".clang-tidy"), "w") as f:
        f.write(clang_tidy_content)
    print("🧹 Generated .clang-tidy")

    # 5. Генерация C++23 Core File (Самодостаточный, БЕЗ PCH)
    # Обрати внимание: IMPLEMENTATION макросы определены прямо здесь.
    cpp_content = f"""/**
 * @file {cpp_filename}
 * @brief Single-file C++23 Project Skeleton (No PCH).
 * Contains pure functions, doctest unit tests, and nanobench benchmarks.
 */

// ============================================================================
// STANDARD INCLUDES
// ============================================================================
#include <iostream>
#include <numeric>
#include <vector>
#include <string>
#include <algorithm>
#include <cmath>
#include <ranges>
#include <expected>
#include <concepts>
#include <fstream>
#include <memory>
#include <optional>

// ============================================================================
// LIBRARIES (Implementation + Interface)
// ============================================================================

#define DOCTEST_CONFIG_IMPLEMENT
#include <doctest/doctest.h>

#define ANKERL_NANOBENCH_IMPLEMENT
#include <nanobench.h>

/// @brief Standard result type for error handling without exceptions.
template<typename T>
using Result = std::expected<T, std::string>;

// ============================================================================
// PURE FUNCTIONAL LOGIC (Level 2: Analysis)
// ============================================================================

/**
 * @brief Calculates the factorial of a non-negative integer using recursion.
 * @tparam T An integral type (int, long, size_t).
 * @param[in] n The number to calculate factorial for.
 * @return Result<T> The factorial value or error if n < 0.
 */
template<std::integral T>
[[nodiscard]] constexpr Result<T> calculate_factorial(T n) noexcept {{
    return (n < 0) ? std::unexpected("Negative input") : Result<T>((n <= 1) ? 1 : n * calculate_factorial(n - 1).value());
}}

/**
 * @brief Filters a collection of numbers based on a predicate.
 * Uses C++20/23 ranges to create a filtered view and materialize it.
 * @param[in] input Vector of integers.
 * @return std::vector<int> A new vector containing only even numbers.
 */
[[nodiscard]] std::vector<int> filter_evens(const std::vector<int>& input) {{
    auto v = input | std::views::filter([](int n){{ return n % 2 == 0; }});
    return std::vector<int>(v.begin(), v.end());
}}

/**
 * @brief Approximates the square root using the Newton-Raphson method.
 * @param[in] x Input value.
 * @param[in] iterations Number of refinement steps.
 * @return double Approximated square root.
 */
[[nodiscard]] constexpr double newton_sqrt(double x, int iterations) noexcept {{
    double g = x / 2.0;
    for(int i=0; i<iterations; ++i) g = (g + x/g) / 2.0;
    return g;
}}

// ============================================================================
// TESTS (Level 3: Creation/Verification)
// ============================================================================

TEST_CASE("Factorial Functionality") {{
    SUBCASE("Happy Path") {{
        CHECK(calculate_factorial(0).value() == 1);
        CHECK(calculate_factorial(1).value() == 1);
        CHECK(calculate_factorial(5).value() == 120);
    }}
    SUBCASE("Edge Cases") {{
        CHECK(calculate_factorial(10).value() > 0); 
    }}
    SUBCASE("Error States") {{
        CHECK(!calculate_factorial(-1).has_value());
        CHECK(calculate_factorial(-5).error() == "Negative input");
    }}
}}

TEST_CASE("Filter Evens Functionality") {{
    SUBCASE("Happy Path") {{
        std::vector<int> in = {{1, 2, 3, 4, 5, 6}};
        auto res = filter_evens(in);
        CHECK(res.size() == 3);
        CHECK(res[0] == 2);
        CHECK(res[2] == 6);
    }}
    SUBCASE("Edge Cases") {{
        CHECK(filter_evens({{}}).empty());
        CHECK(filter_evens({{1, 3, 5}}).empty());
    }}
}}

TEST_CASE("Newton Sqrt Functionality") {{
    CHECK(doctest::Approx(newton_sqrt(25.0, 10)) == 5.0);
    CHECK(doctest::Approx(newton_sqrt(2.0, 10)) == 1.41421356);
}}

// ============================================================================
// BENCHMARKS & EXECUTION
// ============================================================================

void run_benchmarks() {{
    ankerl::nanobench::Bench bench;
    bench.title("Pure Function Benchmarks")
         .unit("ops")
         .warmup(100)
         .relative(true);

    bench.run("Factorial(20)", [] {{
        ankerl::nanobench::doNotOptimizeAway(calculate_factorial(20));
    }});

    std::vector<int> v(1000);
    std::iota(v.begin(), v.end(), 0);
    bench.run("FilterEvens(1000)", [&] {{
        ankerl::nanobench::doNotOptimizeAway(filter_evens(v));
    }});

    bench.run("NewtonSqrt(1M)", [] {{
        ankerl::nanobench::doNotOptimizeAway(newton_sqrt(12345.6789, 20));
    }});

    std::ofstream file("benchmarks.html");
    if (file.is_open()) {{
        bench.render(ankerl::nanobench::templates::htmlBoxplot(), file);
        std::cout << "[Bench] Output written to benchmarks.html\\n";
    }}
}}

int main(int argc, char** argv) {{
    std::cout << "[System] Running Tests...\\n";
    
    doctest::Context context;
    context.applyCommandLine(argc, argv);
    int test_result = context.run();

    if (context.shouldExit()) return test_result;

    if (test_result == 0) {{
        std::cout << "[System] Tests Passed. Running Benchmarks...\\n";
        run_benchmarks();
        
        std::cout << "[System] Opening results...\\n";
        std::system("xdg-open benchmarks.html || open benchmarks.html || echo 'Open benchmarks.html manually'");
    }} else {{
        std::cout << "[System] Tests Failed. Skipping Benchmarks.\\n";
    }}

    return test_result;
}}
"""
    with open(cpp_path, "w") as f:
        f.write(cpp_content)
    print(f"📄 Generated C++ Core: {cpp_path}")
    print("\n✅ Setup Complete! Now you can compile directly.")
import os, subprocess, sys, shutil, webbrowser

root_dir = os.getcwd()

##
# @brief Определяет версию компилятора.
def get_ver(cc):
    try:
        r = subprocess.run([cc, '-dumpfullversion'], capture_output=True, text=True)
        if r.returncode == 0: return r.stdout.strip()
        r = subprocess.run([cc, '--version'], capture_output=True, text=True)
        return r.stdout.split("version ")[1].split()[0] if "version" in r.stdout else "0.0"
    except: return "0.0"

##
# @brief Единый пайплайн с ДЕТАЛЬНЫМИ логами ошибок.
def run_pipeline(cfg):
    ver = get_ver(cfg['cc'])
    suffix = "-cov" if 'cov_flags' in cfg else ""
    v_name = f"{cfg['name']}-{ver}-{cfg['type']}-{cfg['flags'].replace(' ', '_')}{suffix}"
    
    b_dir = os.path.join(root_dir, "build", v_name)
    if not os.path.exists(b_dir): os.makedirs(b_dir); print(f"📂 Created: {v_name}")

    full_cflags = f"{cfg.get('flags', '')} {cfg.get('cov_flags', '')}"
    full_lflags = cfg.get('cov_flags', '')

    # SCons: Подключаем заголовки и исходники
    # Важно: VariantDir дублирует иерархию src в папку build, чтобы не мусорить в src
    sc = f"env=Environment(CXX='{cfg['cc']}');" \
         f"env.Append(CXXFLAGS='{full_cflags}', LINKFLAGS='{full_lflags}');" \
         f"env.Append(CPPPATH=['../../include']);" \
         f"VariantDir('.', '../../src', duplicate=0);" \
         f"env.Program('main', source=Glob('*.cpp'))"
    
    with open(os.path.join(b_dir, "SConstruct"), "w") as f: f.write(sc)

    print(f"🔨 Build: {v_name}...")
    
    # --- ИЗМЕНЕНИЕ: Захват и вывод логов ---
    # Запускаем SCons
    proc = subprocess.run(["scons", "-j4"], cwd=b_dir, capture_output=True, text=True)
    
    if proc.returncode != 0:
        print(f"❌ Build Failed in {b_dir}")
        print("🔻🔻🔻 LOGS START 🔻🔻🔻")
        print(proc.stdout) # Стандартный вывод (обычно ошибки компиляции здесь)
        print(proc.stderr) # Ошибки системы сборки
        print("🔺🔺🔺 LOGS END 🔺🔺🔺")
        return # Прерываем выполнение для этой конфигурации
    # ---------------------------------------
    
    bin_p = os.path.join(b_dir, "main")
    if not os.path.exists(bin_p): return print("❌ Binary missing (Success reported but file not found?)")

    print(f"🚀 Run: {os.path.basename(bin_p)}")
    
    # Запуск бинарника
    run_proc = subprocess.run([bin_p], cwd=b_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if 'cov_flags' not in cfg:
        print(run_proc.stdout) # Показываем вывод программы, если это не тест покрытия
    
    if run_proc.returncode != 0:
        print(f"⚠️ Runtime Error: {run_proc.stderr}")

    # --- COVERAGE ---
    if 'cov_tool' in cfg:
        print("📊 Generating Coverage Report...")
        try:
            info, html = os.path.join(b_dir, "cov.info"), os.path.join(b_dir, "html")
            
            # Флаги для подавления ошибок (GCC 15+ часто требует этого)
            ignore_flags = ['--ignore-errors', 'mismatch,mismatch'] # Удвоенный mismatch как в логе
            rc_flags = ['--rc', 'branch_coverage=1']

            # 1. Сбор данных (Scan)
            # Тут консоль может всё ещё выдать WARNING, это нормально — мы их отфильтруем ниже
            subprocess.run(
                [cfg['cov_tool'], '-c', '-d', b_dir, '-o', info] + rc_flags + ignore_flags, 
                stdout=subprocess.DEVNULL, check=True
            )
            
            # 2. Очистка: ЯВНО удаляем системные файлы (/usr/include и т.д.)
            subprocess.run(
                [cfg['cov_tool'], '--remove', info, '/usr/*', '-o', info] + rc_flags + ignore_flags,
                stdout=subprocess.DEVNULL, check=True
            )

            # 3. Фильтрация: Оставляем только то, что внутри папки src
            subprocess.run(
                [cfg['cov_tool'], '--extract', info, '*/src/*', '-o', info] + rc_flags + ignore_flags,
                stdout=subprocess.DEVNULL, check=True
            )

            # 4. Генерация HTML
            subprocess.run(['genhtml', info, '-o', html, '--branch-coverage'], stdout=subprocess.DEVNULL, check=True)
            
            idx = os.path.join(html, "index.html")
            if os.path.exists(idx): 
                print(f"🌍 Open: {idx}")
                webbrowser.open(f"file://{os.path.abspath(idx)}")
        except Exception as e:
            print(f"⚠️ Coverage Error: {e}")
def gen_docs():
    print("📚 Docs & UML...")
    cfg = """
    PROJECT_NAME           = "CppSconExample"
    OUTPUT_DIRECTORY       = docs
    GENERATE_LATEX         = NO
    HAVE_DOT               = YES
    UML_LOOK               = YES
    CALL_GRAPH             = YES
    EXTRACT_ALL            = YES
    RECURSIVE              = YES
    INPUT                  = .
    EXCLUDE_PATTERNS       = */build/* */.git/* */.ipynb_checkpoints/*
    """

    try:
        subprocess.run(["doxygen", "-"], input=cfg, text=True, stderr=subprocess.DEVNULL, check=True)
        
        path = os.path.join("docs", "html", "index.html")
        if os.path.exists(path):
            print(f"🌍 Open: {path}")
            webbrowser.open(f"file://{os.path.abspath(path)}")
        else:
            print(f"⚠️ Index not found: {path}")
            
    except FileNotFoundError:
        print("❌ Error: Doxygen not installed (try: sudo apt install doxygen graphviz)")
    except subprocess.CalledProcessError:
        print("❌ Error: Doxygen returned non-zero exit code")
def clean(target="all"):
    if target == "all":
        if os.path.exists("build"):
            shutil.rmtree("build")
            print("🧹 Build directory removed.")
    else:
        # Логика удаления конкретной папки (например, по имени варианта)
        pass
def format_code():
    src_dir = "src"
    include_dir = "include"
    extensions = {".cpp", ".h", ".hpp"}

    if os.path.exists(src_dir):
        for root, dirs, files in os.walk(src_dir):
            for file in files:
                if any(file.endswith(ext) for ext in extensions):
                    path = os.path.join(root, file)
                    print(f"✨ Formatting {path}")
                    subprocess.run(["clang-format", "-i", path])

    if os.path.exists(include_dir):
        for root, dirs, files in os.walk(include_dir):
            for file in files:
                if any(file.endswith(ext) for ext in extensions):
                    path = os.path.join(root, file)
                    print(f"✨ Formatting {path}")
                    subprocess.run(["clang-format", "-i", path])


In [2]:
configs = [
    # Обычный Debug
    # {
    #     'name': 'GCC',   
    #     'cc': 'g++',     
    #     'type': 'debug', 
    #     # Добавил -std=c++23
    #     'flags': '-std=c++23 -g -O0'
    # },
    # # Обычный Release
    # {
    #     'name': 'CLANG', 
    #     'cc': 'clang++', 
    #     'type': 'release',
    #     # Добавил -std=c++23
    #     'flags': '-std=c++23 -O3 -Wall'
    # },
    # Coverage Configuration
    {
        'name': 'GCC-Cov',
        'cc': 'g++',
        'type': 'test',
        # Добавил -std=c++23
        'flags': '-std=c++23 -g -O0',
        'cov_flags': '--coverage',
        'cov_tool': 'lcov'
    }
]

for c in configs: run_pipeline(c)

📂 Created: GCC-Cov-15.2.1-test--std=c++23_-g_-O0-cov
🔨 Build: GCC-Cov-15.2.1-test--std=c++23_-g_-O0-cov...
🚀 Run: main
📊 Generating Coverage Report...


Possible precedence problem between ! and pattern match (m//) at /sbin/lcov line 2596.
geninfo: WARNING: Duplicate specification "base-directory|b=s" for option "b"
geninfo: WARNING: geninfo: WARNING: /usr/include/c++/15/bits/allocator.h:168: unexecuted block on non-branch line with non-zero hit count.  Use "geninfo --rc geninfo_unexecuted_blocks=1 to set count to zero.
Possible precedence problem between ! and pattern match (m//) at /sbin/lcov line 2596.
Possible precedence problem between ! and pattern match (m//) at /sbin/lcov line 2596.


🌍 Open: /home/igors/Projects/CppSconExample/build/GCC-Cov-15.2.1-test--std=c++23_-g_-O0-cov/html/index.html


In [ ]:
# project_tdd_setup('.')

In [ ]:
# gen_docs()

In [ ]:
# format_code()

In [ ]:
# clean()